In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
path_meta = 'C:/Users/Sam/Downloads/HO3D_v3/HO3D_v3/HO3D_v3/train/ABF10/meta'
i = np.random.randint(0,len(os.listdir(path_meta)),1).item()
os.path.join(path_meta, os.listdir(path_meta)[i])

In [ ]:
i = np.random.randint(0,len(os.listdir(path_meta)),1).item()
meta = np.load(os.path.join(path_meta, os.listdir(path_meta)[i]), allow_pickle=True)
meta

In [ ]:
from scipy.spatial.transform import Rotation as R
import numpy as np

In [ ]:
r = R.from_rotvec(meta['objRot'][:,0])
m=np.linalg.inv(r.as_matrix())
m=np.concatenate([m,np.zeros((1,3))])
m=np.concatenate([m,np.concatenate([-meta['objTrans'],np.ones(1)])[:,None]],1)
m

In [ ]:
import os
import json
import numpy as np
from scipy.spatial.transform import Rotation as R
import cv2

base_folder = 'C:/Users/Sam/Downloads/HO3D_v3/HO3D_v3/HO3D_v3/train/ABF10'
path_meta = os.path.join(base_folder, 'meta')
path_images = os.path.join(base_folder, 'images')

# Read the intrinsic camera matrix from the metadata
meta = np.load(os.path.join(path_meta, os.listdir(path_meta)[0]), allow_pickle=True)
cam_mat = meta['camMat']

# Extract the intrinsic camera parameters
fl_x = cam_mat[0, 0]
fl_y = cam_mat[1, 1]
cx = cam_mat[0, 2]
cy = cam_mat[1, 2]

# Read the image dimensions
image_files = os.listdir(path_images)
image_path = os.path.join(path_images, image_files[0])
image = cv2.imread(image_path)
h, w, _ = image.shape

# Calculate camera angles (assuming a pinhole camera model)
camera_angle_x = np.arctan2(w - cx, fl_x)
camera_angle_y = np.arctan2(h - cy, fl_y)

# Create the JSON data structure
data = {
    "camera_angle_x": float(camera_angle_x),
    "camera_angle_y": float(camera_angle_y),
    "fl_x": float(fl_x),
    "fl_y": float(fl_y),
    "k1": 0.0,
    "k2": 0.0,
    "k3": 0.0,
    "k4": 0.0,
    "p1": 0.0,
    "p2": 0.0,
    "is_fisheye": False,
    "cx": float(cx),
    "cy": float(cy),
    "w": float(w),
    "h": float(h),
    "aabb_scale": 2,
    "frames": []
}

for meta_file in os.listdir(path_meta):
    meta_path = os.path.join(path_meta, meta_file)
    try:
        meta = np.load(meta_path, allow_pickle=True)

        # Calculate the transformation matrix
        m = cv2.Rodrigues(meta['objRot'])[0].T
        m = np.concatenate([m, np.zeros((1, 3))])
        m = np.concatenate([m, np.concatenate([- np.matmul(meta['objTrans'],cv2.Rodrigues(meta['objRot'])[0]), np.ones(1)])[:, None]], 1)

        # Prepare the frame entry
        frame = {
            "file_path": os.path.join('./images', meta_file.replace('.pkl', '.jpg')),
            "sharpness": 150.,#float(meta['sharpness']),  # Replace with the actual sharpness value calculation
            "transform_matrix": m.tolist()
        }

        # Add the frame entry to the JSON data
        data["frames"].append(frame)
    except Exception as e:
        print(f"Error processing {meta_file}: {str(e)}")

# Write the JSON data to a file
json_file = os.path.join(base_folder, 'transform.json')
with open(json_file, 'w') as f:
    json.dump(data, f, indent=2)

print("JSON file generated successfully!")


In [ ]:
import os
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go

base_folder = 'C:/Users/Sam/Downloads/HO3D_v3/HO3D_v3/HO3D_v3/train/ABF10'
#base_folder = '/path/to/base/folder'

# Path to the meta folder
path_meta = os.path.join(base_folder, 'meta')

# Path to the images folder
path_images = os.path.join(base_folder, 'images')

# Path to the depth folder
path_depth = os.path.join(base_folder, 'depth')

# Depth scale factor
depth_scale = 0.00012498664727900177

# Number of random images to load
n = 2

# Radius for filtering points
radius = 0.5

# List to store the point cloud data for each image
point_clouds = []

# Randomly select n images
meta_files = np.random.choice(os.listdir(path_meta), n)


# Create plotly figure
fig = go.Figure()

# Process each image
for meta_file in meta_files:
    # Load metadata
    meta_path = os.path.join(path_meta, meta_file)
    image_file = os.path.splitext(meta_file)[0] + '.jpg'
    image_path = os.path.join(path_images, image_file)
    depth_file = os.path.splitext(meta_file)[0] + '.png'
    depth_path = os.path.join(path_depth, depth_file)
    
    if not os.path.isfile(meta_path) or not os.path.isfile(image_path) or not os.path.isfile(depth_path):
        continue

    meta = np.load(meta_path, allow_pickle=True)

    # Load camera matrix
    cam_mat = meta['camMat']
    fx = cam_mat[0, 0]
    fy = cam_mat[1, 1]
    cx = cam_mat[0, 2]
    cy = cam_mat[1, 2]

    # Load object rotation and translation
    obj_rot = meta['objRot'][:, 0]
    obj_trans = meta['objTrans']

    # Calculate camera transformation matrix
    #r = R.from_rotvec(obj_rot)
    #m = r.as_matrix()
    m = cv2.Rodrigues(meta['objRot'])[0].T
    m = np.concatenate([m, np.zeros((1, 3))])
    m = np.concatenate([m, np.concatenate([- np.matmul(meta['objTrans'],cv2.Rodrigues(meta['objRot'])[0]), np.ones(1)])[:, None]], 1)

    # Load image
    image = cv2.imread(image_path)

    # Load depth map
    depth_img = cv2.imread(depth_path)
    dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
    dpt = dpt * depth_scale

    # Get image shape
    image_shape = image.shape[:2]

    # Calculate image coordinates (u, v) and depth coordinates (x, y, z)
    u, v = np.meshgrid(np.arange(image_shape[1]), np.arange(image_shape[0]))
    x = (u - cx) * dpt / fx
    y = -(v - cy) * dpt / fy
    z = -dpt



    # Create homogeneous coordinate matrix
    coordinates = np.stack((x, y, z, np.ones_like(x)), axis=2)

    # Transform coordinates to object reference frame
    transformed_coordinates = np.matmul(m, coordinates.reshape(-1, 4).T).T
    print(transformed_coordinates.shape)

    # Extract transformed points and colors
    transformed_points = transformed_coordinates[:, :3].T
    colors = image.reshape(-1, 3) / 255.0

    # Apply filtering based on radius
    distances = np.linalg.norm(transformed_points, axis=0)
    filtered_indices = np.where(distances <= radius)[0]
    filtered_points = transformed_points[:, filtered_indices]
    filtered_colors = colors[filtered_indices]

    print(filtered_points.shape)

    # Add filtered point cloud to the list
    point_clouds.append((filtered_points, filtered_colors))
    fig.add_trace(go.Scatter3d(
        x=filtered_points[0],
        y=filtered_points[1],
        z=filtered_points[2],
        mode='markers',
        marker=dict(
            size=1,
            color=filtered_colors,
            opacity=0.8
        ),
    ))

    # Add object bounding box corners to the plot
    objCorners = meta['objCorners3DRest']
    #objCornersTrans = np.matmul(objCorners, cv2.Rodrigues(meta['objRot'])[0].T) + meta['objTrans']
    corners_x = objCorners[:, 0]
    corners_y = objCorners[:, 1]
    corners_z = objCorners[:, 2]

    fig.add_trace(go.Scatter3d(x=corners_x, y=corners_y, z=corners_z,
                            mode='markers', marker=dict(size=5, color='red')))




# Set layout
fig.update_layout(scene=dict(
    xaxis=dict(range=[-radius, radius]),
    yaxis=dict(range=[-radius, radius]),
    zaxis=dict(range=[-radius, radius]),
    aspectmode='manual',
    aspectratio=dict(x=1, y=1, z=1),
    camera=dict(
        eye=dict(x=0.5, y=0.5, z=0.2),
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0)
    )
))

# Show the animation
fig.show()


In [ ]:
import os
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go

base_folder = 'C:/Users/Sam/Downloads/HO3D_v3/HO3D_v3/HO3D_v3/train/ABF10'
#base_folder = '/path/to/base/folder'

# Path to the meta folder
path_meta = os.path.join(base_folder, 'meta')

# Path to the images folder
path_images = os.path.join(base_folder, 'images')

# Path to the depth folder
path_depth = os.path.join(base_folder, 'depth')

# Depth scale factor
depth_scale = 0.00012498664727900177

# Number of random images to load
n = 2

# Radius for filtering points
radius = 2.

# List to store the point cloud data for each image
point_clouds = []

# Randomly select n images
meta_file = np.random.choice(os.listdir(path_meta), n)[0]

In [ ]:
import plotly.graph_objects as go

# Load metadata
meta_path = os.path.join(path_meta, meta_file)
image_file = os.path.splitext(meta_file)[0] + '.jpg'
image_path = os.path.join(path_images, image_file)
depth_file = os.path.splitext(meta_file)[0] + '.png'
depth_path = os.path.join(path_depth, depth_file)

meta = np.load(meta_path, allow_pickle=True)

# Load camera matrix
cam_mat = meta['camMat']
fx = cam_mat[0, 0]
fy = cam_mat[1, 1]
cx = cam_mat[0, 2]
cy = cam_mat[1, 2]

# Load image
image = cv2.imread(image_path)

# Load depth map
depth_img = cv2.imread(depth_path)
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# Get image shape
image_shape = image.shape[:2]

# Calculate image coordinates (u, v) and depth coordinates (x, y, z)
u, v = np.meshgrid(np.arange(image_shape[1]), np.arange(image_shape[0]))
x = (u - cx) * dpt / fx
y = -(v - cy) * dpt / fy
z = -dpt

# Create homogeneous coordinate matrix
coordinates = np.stack((x, y, z, np.ones_like(x)), axis=2)

m = cv2.Rodrigues(meta['objRot'])[0].T
m = np.concatenate([m, np.zeros((1, 3))])
m = np.concatenate([m, np.concatenate([- np.matmul(meta['objTrans'],cv2.Rodrigues(meta['objRot'])[0]), np.ones(1)])[:, None]], 1)

# Transform coordinates to object reference frame
coordinates = np.matmul(m, coordinates.reshape(-1, 4).T).T

# # Create homogeneous coordinate matrix
# coordinates = np.stack((x, y, z), axis=2)
# #coordinates = coordinates - meta['objTrans']
# coordinates = np.matmul(coordinates,cv2.Rodrigues(meta['objRot'])[0]) - np.matmul(meta['objTrans'],cv2.Rodrigues(meta['objRot'])[0])

print(coordinates.shape)
x = coordinates[...,0]
y = coordinates[...,1]
z = coordinates[...,2]

# Create 3D scatter plot for point cloud
fig = go.Figure(data=go.Scatter3d(x=x.flatten(), y=y.flatten(), z=z.flatten(),
                                 mode='markers', marker=dict(size=1)))

# Add object bounding box corners to the plot
objCorners = meta['objCorners3DRest']
objCornersTrans = np.matmul(objCorners, cv2.Rodrigues(meta['objRot'])[0].T) + meta['objTrans']
corners_x = objCorners[:, 0]
corners_y = objCorners[:, 1]
corners_z = objCorners[:, 2]

fig.add_trace(go.Scatter3d(x=corners_x, y=corners_y, z=corners_z,
                           mode='markers', marker=dict(size=5, color='red')))

# Update plot layout
# Set layout
radius=0.5
fig.update_layout(scene=dict(
    xaxis=dict(range=[-radius, radius]),
    yaxis=dict(range=[-radius, radius]),
    zaxis=dict(range=[-radius, radius]),
    aspectmode='manual',
    aspectratio=dict(x=1, y=1, z=1),
    camera=dict(
        eye=dict(x=-0.5, y=-0.5, z=0.2),
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0)
    )
))
# Show the plot
fig.show()


In [ ]:
import os
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R
import plotly.graph_objects as go

base_folder = 'C:/Users/Sam/Downloads/HO3D_v3/HO3D_v3/HO3D_v3/train/ABF10'
#base_folder = '/path/to/base/folder'

# Path to the meta folder
path_meta = os.path.join(base_folder, 'meta')

# Path to the images folder
path_images = os.path.join(base_folder, 'images')

# Path to the depth folder
path_depth = os.path.join(base_folder, 'depth')

# Depth scale factor
depth_scale = 0.00012498664727900177

# Number of random images to load
n = 2

# Radius for filtering points
radius = 2.

# List to store the point cloud data for each image
point_clouds = []

# Randomly select n images
meta_file = np.random.choice(os.listdir(path_meta), n)[0]

In [ ]:
# Load metadata
meta_path = os.path.join(path_meta, meta_file)
image_file = os.path.splitext(meta_file)[0] + '.jpg'
image_path = os.path.join(path_images, image_file)
depth_file = os.path.splitext(meta_file)[0] + '.png'
depth_path = os.path.join(path_depth, depth_file)

meta = np.load(meta_path, allow_pickle=True)

m = cv2.Rodrigues(meta['objRot'])[0].T
m = np.concatenate([m, np.zeros((1, 3))])
m = np.concatenate([m, np.concatenate([- np.matmul(meta['objTrans'],cv2.Rodrigues(meta['objRot'])[0]), np.ones(1)])[:, None]], 1)

cam_mat = meta['camMat']
fx = cam_mat[0, 0]
fy = cam_mat[1, 1]
cx = cam_mat[0, 2]
cy = cam_mat[1, 2]

# Load image
image = cv2.imread(image_path)

# Get image shape
image_shape = image.shape[:2]

# Load depth map
depth_img = cv2.imread(depth_path)
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale


In [ ]:
from extensions.cameras import Cameras
import torch

cams = Cameras( fx, fy, cx, cy, image_shape[1], image_shape[0], torch.tensor(m[None], dtype=torch.float), trainable=False)

In [ ]:
v, u = torch.meshgrid(torch.arange(image_shape[0]), torch.arange(image_shape[1]))
u=u.flatten()
v=v.flatten()

In [ ]:
u.dtype

In [ ]:
rays = cams.get_rays(torch.zeros_like(u).long(),u.float(),v.float())

In [ ]:
pts = rays.origins + rays.dirs*dpt.flatten()[:,None]

In [ ]:
pts.shape

In [ ]:
fig = go.Figure(data=go.Scatter3d(x=pts[:,0], y=pts[:,1], z=pts[:,2],
                                 mode='markers', marker=dict(size=1, color=image.reshape(-1,3))))

# Add object bounding box corners to the plot
objCorners = meta['objCorners3DRest']
objCornersTrans = np.matmul(objCorners, cv2.Rodrigues(meta['objRot'])[0].T) + meta['objTrans']
corners_x = objCorners[:, 0]
corners_y = objCorners[:, 1]
corners_z = objCorners[:, 2]

fig.add_trace(go.Scatter3d(x=corners_x, y=corners_y, z=corners_z,
                           mode='markers', marker=dict(size=5, color='red')))

# Update plot layout
# Set layout
radius=0.5
fig.update_layout(scene=dict(
    xaxis=dict(range=[-radius, radius]),
    yaxis=dict(range=[-radius, radius]),
    zaxis=dict(range=[-radius, radius]),
    aspectmode='manual',
    aspectratio=dict(x=1, y=1, z=1),
    camera=dict(
        eye=dict(x=-0.5, y=-0.5, z=0.2),
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0)
    )
))
# Show the plot
fig.show()

In [ ]:
cams.poses